In [98]:
from IPython.core.display import display, HTML
from tqdm import tqdm
import pandas as pd
import re
pd.set_option('display.max_rows', 100)

In [2]:
from horacy import HoracyModel
model = HoracyModel('model_all/').load()

load done in 12.05 seconds


In [150]:
def score_text(text, criteria):
    total = 0
    value = 1
    for c in criteria:
        if type(c) in (int,float):
            value = c
        else:
            c = c.replace('_',r'\b')
            total += value if re.findall(c,text,re.I) else 0
    return total

def score_results(i_d_lists, criteria):
    results = []
    for i,d in zip(*i_d_lists):
        doc = model.get_doc(i)
        text = model.doc_to_text(doc).replace('\n',' ').replace('\r',' ')
        s = score_text(text, criteria)
        text = highlight(text, criteria)
        rec = s,d,i,text,doc
        results += [rec]
    results.sort(key=lambda x:(-x[0],x[1]))
    return results

def highlight(text, criteria, before='<b>', after='</b>'):
    for c in criteria:
        if type(c) in (int,float):
            continue
        else:
            c = '('+c.replace('_',r'\b')+'\w*)'
            text = re.sub(c, before+r'\1'+after, text, flags=re.I)
    return text


def agg_scores(results):
    scores = [x[0] for x in results]
    scores.sort(reverse=True)
    return max(scores),sum(scores),sum([x*x for x in scores])/len(scores),min(scores)


# Outcomes data for COVID-19 after mechanical ventilation adjusted for age

In [157]:
criteria = [5,'mechanical','ventilat',2,'_age','years','_old',1,'\d+ patients','adjust','surviv','discharge','extubate','died','dead','death','mortality','duration','_day','probability','Kaplan-Meier','curve','figure','table']
criteria = [5,'mechanical','ventilat',2,'adjust','_age','years','_old',1,'\d+ patients','surviv','discharge','extubate','died','dead','death','mortality','duration','_day','figure','table']

for query in [
        'results after mechnical ventilation discharged dead died',
        'results after mechnical ventilation discharged dead died survived survivors adjusted age years old',
        'results after mechnical ventilation discharged died survived survivors extubated adjusted',
        'results after mechnical ventilation discharged dead died survived survivors adjusted age years old',
        'results after mechnical ventilation discharged died survived extubated adjusted',
        'results after mechnical ventilation discharged died survived extubated adjusted age',
        'outcomes after mechnical ventilation discharged died survived extubated adjusted',
        'results outcomes after mechnical ventilation discharged died survived extubated adjusted age',
        'results outcomes after mechnical ventilation discharged died survived extubated adjusted',
]:
    q = model.text_to_dense(query)
    i_d = model.dense_ann_query(q,50)
    results = score_results(i_d, criteria)
    scores = agg_scores(results)
    print(scores)


(21, 625, 169.26, 2)
(10, 266, 31.88, 2)
(21, 673, 201.18, 1)
(10, 266, 31.88, 2)
(21, 683, 198.06, 6)
(21, 483, 120.7, 1)
(21, 674, 204.48, 1)
(21, 580, 163.56, 2)
(21, 690, 212.76, 1)


In [158]:
q = model.text_to_dense('results after mechnical ventilation discharged dead died')
q = model.text_to_dense('results after mechnical ventilation discharged survived died')
q = model.text_to_dense('results after mechnical ventilation discharged dead died survived survivors')
q = model.text_to_dense('results after mechnical ventilation discharged dead died survived survivors adjusted age years old')
q = model.text_to_dense('results after mechnical ventilation discharged died survived survivors extubated adjusted')
q = model.text_to_dense('results outcomes after mechnical ventilation discharged died survived extubated adjusted age')
i_d_lists = model.dense_ann_query(q,50)

results = score_results(i_d_lists, criteria)
print(agg_scores(results))
for s,d,i,text,doc in results:
    display(HTML(f"{s} :: {d} :: {i}<br>{text}"))




(21, 580, 163.56, 2)
